In [18]:
import pandas as pd
from sklearn.model_selection import train_test_split
import torch
from torch.utils.data import Dataset, DataLoader
import torch.nn as nn
import torch.optim as optim
import matplotlib.pyplot as plt

In [19]:
from tensorflow import keras
(X_train,y_train),(X_test,y_test) = keras.datasets.mnist.load_data()
X_test.shape

(10000, 28, 28)

In [20]:
X_train = X_train[:8000]
y_train = y_train[:8000]
X_test = X_train[:2000]
y_test = y_train[:2000]

In [21]:
X_train = X_train/255.0
X_test = X_test/255.0

In [22]:
class CustomDataset(Dataset):

  def __init__(self, features, labels):

    # Convert to PyTorch tensors
    self.features = torch.tensor(features, dtype=torch.float32).reshape(-1,1,28,28)
    self.labels = torch.tensor(labels, dtype=torch.long)

  def __len__(self):
    return len(self.features)

  def __getitem__(self, index):
    return self.features[index], self.labels[index]

In [23]:
train_dataset = CustomDataset(X_train, y_train)

In [24]:
test_dataset = CustomDataset(X_test, y_test)

In [25]:
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True, pin_memory=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False, pin_memory=True)

In [26]:
class MyNN(nn.Module):
    def __init__(self, input_features):
        super().__init__()

        self.features = nn.Sequential(
            nn.Conv2d(input_features, 32, kernel_size=3, padding='same'),
            nn.ReLU(),
            nn.BatchNorm2d(32),
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.Conv2d(32, 64, kernel_size=3, padding='same'),
            nn.ReLU(),
            nn.BatchNorm2d(64),
            nn.MaxPool2d(kernel_size=2, stride=2)
        )
        self.classifier = nn.Sequential(
            nn.Flatten(),
            nn.Linear(64*7*7, 128),
            nn.ReLU(),
            nn.Dropout(p=0.4),

            nn.Linear(128, 64),
            nn.ReLU(),
            nn.Dropout(p=0.4),

            nn.Linear(64, 10)
        )

    def forward(self, x):
        x = self.features(x)
        x = self.classifier(x)

        return x

In [27]:
learning_rate = 0.01
epochs = 100

In [28]:
model = MyNN(1)
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=learning_rate, weight_decay=1e-4)

In [29]:
# training loop

for epoch in range(epochs):

  total_epoch_loss = 0

  for batch_features, batch_labels in train_loader:

    # forward pass
    outputs = model(batch_features)

    # calculate loss
    loss = criterion(outputs, batch_labels)

    # back pass
    optimizer.zero_grad()
    loss.backward()

    # update grads
    optimizer.step()

    total_epoch_loss = total_epoch_loss + loss.item()

  avg_loss = total_epoch_loss/len(train_loader)
  print(f'Epoch: {epoch + 1} , Loss: {avg_loss}')


Epoch: 1 , Loss: 1.011939345240593
Epoch: 2 , Loss: 0.33111154979467394
Epoch: 3 , Loss: 0.21077364306151866
Epoch: 4 , Loss: 0.1671339184641838
Epoch: 5 , Loss: 0.12967021519318223
Epoch: 6 , Loss: 0.1065255375355482
Epoch: 7 , Loss: 0.0889388890489936
Epoch: 8 , Loss: 0.07918519077450037
Epoch: 9 , Loss: 0.06330596284382045
Epoch: 10 , Loss: 0.056421363950707015
Epoch: 11 , Loss: 0.05537130660098046
Epoch: 12 , Loss: 0.04243813635315746
Epoch: 13 , Loss: 0.036730385243427005
Epoch: 14 , Loss: 0.03602072841557674
Epoch: 15 , Loss: 0.0330688024898991
Epoch: 16 , Loss: 0.030326899180654437
Epoch: 17 , Loss: 0.024276852257549764
Epoch: 18 , Loss: 0.025946934797335418


KeyboardInterrupt: 

In [ ]:
model.eval()

In [ ]:
# evaluation on test data
total = 0
correct = 0

with torch.no_grad():

  for batch_features, batch_labels in test_loader:

    outputs = model(batch_features)

    _, predicted = torch.max(outputs, 1)

    total = total + batch_labels.shape[0]

    correct = correct + (predicted == batch_labels).sum().item()

print(correct/total)

In [ ]:
# evaluation on training data
total = 0
correct = 0

with torch.no_grad():

  for batch_features, batch_labels in train_loader:

    outputs = model(batch_features)

    _, predicted = torch.max(outputs, 1)

    total = total + batch_labels.shape[0]

    correct = correct + (predicted == batch_labels).sum().item()

print(correct/total)